# 한글전처리 및 Tokenization
+ 본과정에서는 한글문장에 대해 불필요한 특수문자, 이메일, 다수 공백등을 제거하는 전처리 수행후에 토큰화 해 보겠습니다.
+ 읽어들인 문장들을 전처리 : 한글, 영문, 숫자, 공백 이외는 제거
+ 전처리된 문장들을 한글 형태소 분석기 이용하여 Tokenize

### 학습목차
1. 한글 토크나이징 Konlpy 설치
2. A방송사 댓글 파일 읽어오기
3. 한글 전처리 :특수문자, 이메일, URL 제거하고 한글, 영문, 숫자, 공백 이외는 제거
4. KoNLPy 형태소 분석기 사용하여 단어 토큰화
5. kss 사용하여 문장 단위 토큰화

## 1. 한글 토크나이징 Konlpy 설치

In [1]:
# Konlpy : 한국어 정보처리를 위한 파이썬 패키지, 형태소 기반 토크나이징
# Konlpy 패키지 안에 Hannanum, Kkma, Komoran, Mecab, Okt 형태소 분류기 있다.

!pip install konlpy

## 2. A방송사 댓글 파일 읽어오기

In [2]:
# pandas read_excel 사용시 필요 라이브러리
!pip install openpyxl

import os
import re
import pandas as pd

     |████████████████████████████████| 242 kB 26.2 MB/s            


In [3]:
# A_comment_train.xlsx A방송사 댓글 다운로드 및 Pandas read_excel() 함수 활용하여 읽어오기
commnet = pd.read_excel('https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_train.xlsx', engine='openpyxl')
commnet.tail()

,Unnamed: 0,data,label
246,246,영상F서비스로 간편하게 설치!좋아요\n우리 회사화이팅!,긍정
247,247,모든 업무에서 맡은바 업무에 서 최선을 다하는 모습이 좋습니다! 화이팅 입니다.,긍정
248,248,"사내방송 특성상 최근 이슈화 되거나, 언급이 자주되는 '키워드'를 중심으로 뉴스를 ...",부정
249,249,방송 시간이 너무 길어요.,부정
250,250,"처음 들어보는 말들이 많은데,, 설명이 없어서 힘드네요.",부정


In [4]:
# 리스트 형태로 데이터 만들기

sent_list = list(commnet.data[:100].values)
sent_list[:2]

['재미는 있는데 시간이 짧은게 아쉽네요~', 'OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄']

## 3. 한글 전처리
- 특수문자, 이메일, URL 제거
- 많은 공백은 1개의 공백으로
- 숫자는 'NUM' 표현
- 한글, 영문, 숫자, 공백 이외는 제거

In [5]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

EMAIL_PATTERN = re.compile(r'''(([a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4}))''')
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)

In [6]:
def cleansing_other(sentence) :
    """
    # 문장을 전처리 (이메일, URL, 공백 등 제거) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub(EMAIL_PATTERN, ' ', sentence)
    sentence = re.sub(URL_PATTERN, ' ', sentence)
    sentence = re.sub(MULTIPLE_SPACES, ' ', sentence)
    sentence = sentence.replace(", )", "")
    
    return sentence

In [7]:
def cleansing_numbers(sentence) :
    """
    # 숫자를 전처리(delexicalization) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

In [8]:
def cleansing_special(sentence) :
    """
    # 특수문자를 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

In [9]:
def preprocess_sent(sentence) :
    """
    # 모든 전처리를 수행 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sent_clean = sentence
    sent_clean = cleansing_other(sent_clean)
    sent_clean = cleansing_special(sent_clean)
    sent_clean = cleansing_numbers(sent_clean)
    sent_clean = re.sub('\s+', ' ', sent_clean)

    return sent_clean

In [10]:
# 한글 샘플 내용을 1개 가져와 전처리 수행 

sample_sentence = sent_list[5]
print(sample_sentence)

너무 재밌게 잘봤습니다~! ㅋㅋ
송OO씨가 출근하자마자 안마의자 앉아서 쉬는게 부럽기도(?) 했네요
쇼호스트분들의 진행솜씨또한 감탄하고 갑니다. 다음은 어디 외부 탐방 예정인가요?!


In [11]:
clean_sentence = preprocess_sent(sample_sentence)
clean_sentence

'너무 재밌게 잘봤습니다 송OO씨가 출근하자마자 안마의자 앉아서 쉬는게 부럽기도 했네요 쇼호스트분들의 진행솜씨또한 감탄하고 갑니다 다음은 어디 외부 탐방 예정인가요'

In [12]:
# 여러개의 한글 샘플 전처리 수행

clean_sentences = []

for sample_sentence in sent_list:
  clean_sentence = preprocess_sent(sample_sentence)
  clean_sentences.append(clean_sentence)

clean_sentences[:5]

['재미는 있는데 시간이 짧은게 아쉽네요',
 'OO 관련 내용은 우리 직원과는 거리가 멀었음 특히 사내에 홍보할 내용은 아니라고 봄',
 '스토리가 너무 딱딱해서 별로였음',
 '프로그램A 화이팅하세요',
 '높은 곳에 올라가는 모습이 너무 위험해 보여요']

## 4. KoNLPy 형태소 분석기 사용하여 단어 토큰화
- 종류 : Okt, Komoran, Hannanum, Kkma

In [13]:
from konlpy.tag import Okt, Komoran, Hannanum, Kkma

In [14]:
# 여러가지 형태소 분석기중에서 선택할수 있도록 하는 함수

def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    else:
        # "kkma":
        tokenizer = Kkma()
        
    return tokenizer

In [15]:
# 선택한 형태소 분석기로 형태소 분석 및 품사태깅을 선택적으로 수행함.

def tokenize(tokenizer_name, original_sent, pos=False):
    tokenizer = get_tokenizer(tokenizer_name)
    sentence = original_sent.replace('\n', '').strip()
    if pos:
        tokens = tokenizer.pos(sentence)
        tokens = [morph + "/" + tag for morph, tag in tokens]
    else:
        tokens = tokenizer.morphs(sentence)
        
    tokenized_sent = ' '.join(tokens)
    
    return tokenized_sent


In [16]:
text = "아버지가방에들어가신다"

print(tokenize("komoran", text, pos=True))
print(tokenize("komoran", text, pos=False))

아버지/NNG 가방/NNP 에/JKB 들어가/VV 시/EP ㄴ다/EC
아버지 가방 에 들어가 시 ㄴ다


In [17]:
text = "아버지가방에들어가신다"

print(tokenize("okt", text, pos=True))
print(tokenize("okt", text, pos=False))

아버지/Noun 가방/Noun 에/Josa 들어가신다/Verb
아버지 가방 에 들어가신다


In [18]:
# 전처리 된 한글 샘플을 OKT 형태소 분석기로 토큰화함
print(clean_sentence)
print('-'*140)
print(tokenize("okt", clean_sentence, pos=True))

실제로 있을수있는 내용이네요 고객 니즈가 잘소통되었네요
--------------------------------------------------------------------------------------------------------------------------------------------
실제/Noun 로/Josa 있을수있는/Adjective 내/Determiner 용이/Noun 네/Suffix 요/Josa 고객/Noun 니즈/Noun 가/Josa 잘/Verb 소통/Noun 되었네요/Verb


In [19]:
# 위의 함수를 사용하지 않고 자체 Okt 형태소 분석기 사용하기 

okt = Okt()

print('OKT 형태소 분석 :',okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))  # 형태소 분류
print('OKT 품사 태깅 :',okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))  # POS 분류
print('OKT 명사 추출 :',okt.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))  # 명사 분류

OKT 형태소 분석 : ['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']
OKT 품사 태깅 : [('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('가봐요', 'Verb')]
OKT 명사 추출 : ['코딩', '당신', '연휴', '여행']


## 배운 내용 정리
1. A방송사 댓글 데이터 가져와 전처리 수행
2. 전처리된 한글 데이터를 KoNLPy OKT 형태소분석기를 통해 토큰화함
3. 전처리 함수를 필요할때 가져다 쓰면 된다.